In [1]:
import pyspark
import numpy as np

try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [2]:
# trip_summary = (driver_id ,car_id ,trip_id ,customer_id ,date(YYYYMMDD), distance_traveled)
# Hallar el conductor con mayor distancia de viaje promedio en Abril de 2016

In [11]:
def random_ids():
    ids = np.random.randint(1,51)
    return (ids, ids)

def random_customer_id():
    return np.random.randint(1, 10000)

def random_date():
    year = np.random.randint(2010, 2020)
    month = np.random.randint(1, 13)
    day = np.random.randint(1, 29)
    return str(year)+'-'+str(month)+'-'+str(day)
    
def random_distance():
    return np.random.randint(400, 3000)

trip_summary = []
for i in range(30000):
    trip_summary.append((random_ids(), i, random_customer_id(), random_date(), random_distance()) )
    
trip_summary_rdd = sc.parallelize(trip_summary)\
                     .map(lambda x: (x[0][0], x[0][1], x[1], x[2], x[3], x[4]))
  
trip_summary_rdd.take(5)

[(28, 28, 0, 4909, '2012-1-20', 1274),
 (46, 46, 1, 8995, '2012-5-17', 2279),
 (47, 47, 2, 7937, '2010-12-24', 1248),
 (42, 42, 3, 4135, '2015-11-4', 2498),
 (41, 41, 4, 969, '2014-9-27', 462)]

In [23]:
mayor_distancia_promedio = trip_summary_rdd.filter(lambda x: x[4].startswith('2016-4'))\
                                           .map(lambda x: (x[0], (x[5], 1)))\
                                           .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))\
                                           .map(lambda x: (x[0], x[1][0]/x[1][1]))\
                                           .top(1, key=lambda x: x[1])

print("El chofer con mayor distancia promedio fue el #%i con %im recorridos" % mayor_distancia_promedio[0])

El chofer con mayor distancia promedio fue el #15 con 2837m recorridos
